### About

**NOTE: This notebook is a continuation of my previous notebook which contained EDA.**   

You can refer the following links:  
- Part-1 EDA: LINK  
- End-to-End implementation with deployment on AWS: https://github.com/Sharma-Ayush/Credit-Card-Fraud-Detection.git

### Objective of this notebook

The objective for this notebook is:  
- Feature Engineering.  
- Handling Imbalance.  
- Feature importance.  
- Cross-validation and hyper parameter tuning of classification models.  
- Testing on test set.

<u>PS:</u>  
- Feel free to contact me if you have any doubts or feedback through the comment section or my socials.
- Please upvote the notebook if you like it, as it would motivate me to develop more projects like these.

### My Socials

Follow me on these platforms, for more such content:  

LinkedIn: https://www.linkedin.com/in/ayush-sharma-660831125/  
Github: https://github.com/Sharma-Ayush  
Kaggle: https://www.kaggle.com/ayushsharma0812

### Import Required Libraries

In [1]:
# Data Mainpulation
import numpy as np
import pandas as pd

### Loading the dataset

In [2]:
df_train = pd.read_csv('Data/creditcard_train.csv')

### References

- https://www.youtube.com/@krishnaik06  
- Hands on Machine Learning with Scikit-Learn & TensorFlow by Aurélien Géron (O'Reilly). CopyRight 2017 Aurélien Géron